<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Ignacio Villagrán

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**


¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo? Por que es un método que busca predecir el rating que un usuario asignará a un item en base a su relación con otros usuarios similares (en esta caso items y rating de usuarios).

¿está basado en el usuario o en los items? ¿Por qué?
Está basado en ítems, ya que compara la valoración (rating) que le ha asignado distintos usuarios a un mismo ítem.

¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
Utiliza los ranking de items en común entre usuarios para sacar desviación promedio y luego realiza una ponderación en base a la cantidad de usuarios que otorgan las valoraciones. 

¿Qué pasaría si se agrega un nuevo rating a la base de datos?
Debiera cambiar la predicción realizada inmediatamente ya que el algoritmo está descrito para que se compute una vez y luego se vaya actualizando según la nueva información.

¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings? Dado que se basa en el promedio, el método funciona con poco feedback del usuario.

# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    499      0 --:--:-- --:--:-- --:--:--   499
100 1546k  100 1546k    0     0  1018k      0  0:00:01  0:00:01 --:--:-- 2885k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    790      0 --:--:-- --:--:-- --:--:--   790
100  385k  100  385k    0     0   371k      0  0:00:01  0:00:01 --:--:--  125M


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    765      0 --:--:-- --:--:-- --:--:--   765
100  230k  100  230k    0     0   223k      0  0:00:01  0:00:01 --:--:--  112M


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 194kB 2.7MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [10]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [11]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**
El train set se utiliza para armar el modelo antes de probarlo con el los datos de test. En este caso el método train se utiliza para sacar el promedio de las diferencias de la base de datos de entrenamiento. 

## Paso 6:

Llego la hora de predecir el rating.

In [12]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [13]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [14]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [15]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** El resultado de la predicción es diferente al rating dado por el usuario ya que se basa en las valoraciones que han dado otros usuarios similares al los mismos items ponderado según número de valoraciones. Por eso existe una diferencia. 

In [16]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [17]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [18]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Si existen diferencias en la recomendación (Utilicé usuario 45)

In [21]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("45", 5)
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1064', '1651', '1650', '1645', '1642']


## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

In [28]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("44", 5)
print('Slope One recommendation for user 44:', reclist_slopeone)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

#most popular
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)
most_popular.train()
top_n = 5

recommendList, maprec, ndcg = most_popular.testrec(input_file='u1.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=top_n,
                                          relevance_threshold=2,
                                          includeRated=False)
user_id = 44

ranking = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]
print('Most Popular recommendation for user {}: {}'.format(user_id, ranking))

Slope One recommendation for user 44: ['1656', '1064', '1643', '1642', '1625']
Most Popular recommendation for user 44: [50, 100, 181, 286, 1]
